In [ ]:
!wandb login $wandb_api_key
import wandb
wandb.login()
wandb.init(project="huggingface",name="finbert-augmented2")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Currently logged in as: abhinavdasu0412 (abhinavdasu0412-pes-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [ ]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.utils.class_weight import compute_class_weight
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import (
    AutoTokenizer,
    AutoConfig,
    BertPreTrainedModel,
    BertModel,
    Trainer,
    TrainingArguments,
    AutoModel,
    EarlyStoppingCallback
)
import torch.nn as nn

# Load augmented dataset
df = pd.read_csv("augmented_financial_news.csv")

# Check for essential columns
assert 'combined' in df.columns and 'Sentiment' in df.columns, "Expected columns not found!"

# Capitalize sentiment to match mapping
df['Sentiment'] = df['Sentiment'].str.capitalize()

# Label mapping
label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
df = df[df['Sentiment'].isin(label_map.keys())]
df['label'] = df['Sentiment'].map(label_map)

# Train-test split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['combined'].tolist(), df['label'].tolist(),
    test_size=0.2, stratify=df['label'], random_state=42
)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=256)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=256)

# Dataset class
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_encodings, train_labels)
val_dataset = NewsDataset(val_encodings, val_labels)

# Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# Custom FinBERT model
class CustomFinBERT(nn.Module):
    def __init__(self, model_name, num_labels, class_weights=None, freeze_until_layer=9):
        super().__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        
        for name, param in self.bert.named_parameters():
            if 'encoder.layer' in name:
                layer_num = int(name.split('.')[2])
                if layer_num < freeze_until_layer:
                    param.requires_grad = False
            elif 'embeddings' in name:
                param.requires_grad = False
            elif 'pooler' in name:
                param.requires_grad = False

        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, num_labels)
        self.class_weights = class_weights
        self.softmax = nn.Softmax(dim=-1)  # Added softmax layer

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        logits = self.classifier(self.dropout(pooled_output))

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            loss = loss_fct(logits, labels)
        output_logits = self.softmax(logits) if labels is None else logits

        return SequenceClassifierOutput(
            loss=loss,
            logits=output_logits,  # These will be softmax probs at inference time
            hidden_states=None,
            attentions=None
        )


# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Config and model init
config = AutoConfig.from_pretrained("ProsusAI/finbert", num_labels=3)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class_weights = class_weights.to(device)

model = CustomFinBERT(model_name="ProsusAI/finbert", num_labels=3, class_weights=class_weights)
model.to(device)

# Training arguments
training_args = TrainingArguments(
    output_dir="./finbert-results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    num_train_epochs=7,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    save_total_limit=2,
    logging_dir="./logs",
)

# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
)

# Start training
trainer.train()

/tmp/ipykernel_31/1969434352.py:141: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.649300,0.553953,0.768188,0.766651,0.775453,0.768188
2,0.446300,0.392842,0.852782,0.853503,0.855069,0.852782
3,0.309900,0.319240,0.892011,0.891574,0.891775,0.892011
4,0.221300,0.304298,0.912696,0.912311,0.912835,0.912696
5,0.169100,0.303336,0.921541,0.921234,0.921211,0.921541
6,0.135600,0.303937,0.929815,0.929707,0.929775,0.929815
7,0.113400,0.305467,0.932097,0.932013,0.932040,0.932097


TrainOutput(global_step=12271, training_loss=0.29212026135203173, metrics={'train_runtime': 2699.8986, 'train_samples_per_second': 72.696, 'train_steps_per_second': 4.545, 'total_flos': 0.0, 'train_loss': 0.29212026135203173, 'epoch': 7.0})

In [4]:
import torch
model_path="finbert.pth"
torch.save(model.state_dict(),model_path)

artifact=wandb.Artifact(name="finbert-augmented",type="model")
artifact.add_file(model_path)
wandb.log_artifact(artifact)

<Artifact finbert-augmented>